In [3]:
%load_ext tensorboard
#https://medium.com/@shaolinkhoa/install-tensorflow-gpu-2-0-alpha-on-anaconda-for-windows-10-ubuntu-ced099010b21

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version

import tensorflow as tf
import os
from tensorflow import keras
# import keras

import numpy as np

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0-beta1


In [5]:
data_size = 1000
# 80% of the data is for training.
train_pct = 0.8

train_size = int(data_size * train_pct)

# Create some input data between -1 and 1 and randomize it.
x = np.linspace(-1, 1, data_size)
np.random.shuffle(x)

# Generate the output data.
# y = 0.5x + 2 + noise
y = 0.5 * x + 2 + np.random.normal(0, 0.05, (data_size, ))

# Split into test and train pairs.
x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]

In [10]:
print(model.predict([60, 25, 2]))

[[32.002335 ]
 [14.502413 ]
 [ 3.0024674]]


In [7]:
# logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join("logs", "scalars", datetime.now().strftime("%Y%m%d-%H%M%S"))

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = keras.models.Sequential([
    keras.layers.Dense(16, input_dim=1),
    keras.layers.Dense(1),
])

model.compile(
    loss='mse', # keras.losses.mean_squared_error
    optimizer=keras.optimizers.SGD(lr=0.2),
    metrics=["accuracy"]
)

print("Training ... With default parameters, this takes less than 10 seconds.")
training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size=train_size,
    verbose=1, # Suppress chatty output; use Tensorboard instead
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback],
)

print("Average test loss: ", np.average(training_history.history['loss']))

Training ... With default parameters, this takes less than 10 seconds.
Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 0s 428us/sample - loss: 4.3546 - accuracy: 0.0000e+00 - val_loss: 0.1241 - val_accuracy: 0.0000e+00
Epoch 2/100
800/800 [==============================] - 0s 90us/sample - loss: 0.1198 - accuracy: 0.0000e+00 - val_loss: 0.0309 - val_accuracy: 0.0000e+00
Epoch 3/100
800/800 [==============================] - 0s 10us/sample - loss: 0.0322 - accuracy: 0.0000e+00 - val_loss: 0.0096 - val_accuracy: 0.0000e+00
Epoch 4/100
800/800 [==============================] - 0s 9us/sample - loss: 0.0083 - accuracy: 0.0000e+00 - val_loss: 0.0037 - val_accuracy: 0.0000e+00
Epoch 5/100
800/800 [==============================] - 0s 10us/sample - loss: 0.0037 - accuracy: 0.0000e+00 - val_loss: 0.0031 - val_accuracy: 0.0000e+00
Epoch 6/100
800/800 [==============================] - 0s 9us/sample - loss: 0.0026 - accuracy: 0.0000e+00 - val_

Epoch 54/100
800/800 [==============================] - 0s 10us/sample - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0025 - val_accuracy: 0.0000e+00
Epoch 55/100
800/800 [==============================] - 0s 11us/sample - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0025 - val_accuracy: 0.0000e+00
Epoch 56/100
800/800 [==============================] - 0s 19us/sample - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0025 - val_accuracy: 0.0000e+00
Epoch 57/100
800/800 [==============================] - 0s 12us/sample - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0025 - val_accuracy: 0.0000e+00
Epoch 58/100
800/800 [==============================] - 0s 9us/sample - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0025 - val_accuracy: 0.0000e+00
Epoch 59/100
800/800 [==============================] - 0s 12us/sample - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 0.0025 - val_accuracy: 0.0000e+00
Epoch 60/100
800/800 [==============================] - 0s 12us/sample 

In [8]:
logdir = os.path.join("logs", "scalars", datetime.now().strftime("%Y%m%d-%H%M%S"))
file_writer = tf.summary.create_file_writer(os.path.join(logdir, "metrics"))
file_writer.set_as_default()

def lr_schedule(epoch):
  """
  Returns a custom learning rate that decreases as epochs progress.
  """
  learning_rate = 0.2
  if epoch > 10:
    learning_rate = 0.02
  if epoch > 20:
    learning_rate = 0.01
  if epoch > 50:
    learning_rate = 0.005

  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = keras.models.Sequential([
    keras.layers.Dense(16, input_dim=1),
    keras.layers.Dense(1),
])

model.compile(
    loss='mse', # keras.losses.mean_squared_error
    optimizer=keras.optimizers.SGD(),
)

training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size=train_size,
    verbose=0, # Suppress chatty output; use Tensorboard instead
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback, lr_callback],
)

In [9]:
print(model.predict([60, 25, 2]))

[[32.002335 ]
 [14.502413 ]
 [ 3.0024674]]
